In [1]:
import numpy as np
import csv
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
#from tensorflow.examples.tutorials.mnist import input_data
from torch.autograd import Variable
import sklearn.metrics as metrics
import pickle
import os

### Using 10 fold cross validation 

In [2]:
path = os.getcwd() + '/bioassay-datasets/'
p_fingerprints = []
c_fingerprints = []
labels = []
with open(path+'AID362red_train.csv') as csvfile:
    readcsv = csv.reader(csvfile)
    for row in readcsv:
        p_fingerprints.append(row[:112])
        c_fingerprints.append(row[112:-1])
        labels.append(row[-1])

In [3]:
p_fingerprints = np.asarray(p_fingerprints)[1:]
p_fingerprints = p_fingerprints.astype(int)

c_fingerprints = np.asarray(c_fingerprints)[1:]
c_fingerprints = c_fingerprints.astype(float)

#Normalise the features
c_fingerprints = (c_fingerprints - np.mean(c_fingerprints,axis=0))/np.std(c_fingerprints,axis=0)

#p2_fingerprints = np.ones(p_fingerprints.shape)
(no_examples , ip_dim) = p_fingerprints.shape
ip_dim2 = c_fingerprints.shape[1]
labels = labels[1:]
print(no_examples,ip_dim)
print(ip_dim2)
p_fingerprints[(p_fingerprints==0)] = -1


(3423, 112)
32


In [4]:
labels2 = np.zeros((len(labels),1))
for i,l in enumerate(labels):
    if l=='Active':
        labels2[i] = 1
    else:
        labels2[i] = 0
labels2 = labels2.astype(int)
total_pos = np.sum(labels2)
print("tot_positive",total_pos)
fp_allowed = int((no_examples-total_pos)*0.05)
print("fp allowed",fp_allowed)

('tot_positive', 48)
('fp allowed', 168)


#####  Rnadomly permute the data

In [5]:
ind = np.arange(no_examples)
np.random.shuffle(ind)
p_fingerprints = p_fingerprints[ind]
c_fingerprints = c_fingerprints[ind]
labels2 = labels2[ind]

In [6]:
# def get_train_batch(batch_size,binary=True):
#     samples = np.random.randint(low=0,high=no_examples,size=(batch_size,1))
#     if binary == True:
#         train_batch = p_fingerprints[samples].reshape(batch_size,ip_dim)
#         train_batch = train_batch.astype(int)
#     else:
#         train_batch = c_fingerprints[samples].reshape(batch_size,ip_dim2)
#         train_batch = train_batch.astype(float)
    
#     train_batch = torch.cuda.FloatTensor(train_batch)
#     train_batch = Variable(train_batch,requires_grad=False).cuda()
#     #print(labels2[samples])
#     target = Variable(torch.cuda.LongTensor(labels2[samples]),requires_grad=False)
#     target = target.view(batch_size,)
#     #print(target.type)
#     return train_batch,target

In [7]:
no_active_ele = (sum(labels2))
print(no_active_ele)
batch_size = 128

[48]


In [8]:
def get_train_batch(batch_size,validation_iter=0,binary=True):
    
    if validation_iter == 0: #no validation
        curr_data_size = no_examples
        labels_train = labels2
    else:
        curr_data_size = int(no_examples*0.8)
        interval_size = int(no_examples*0.2)
        
        if(val_iter==1):
            s_ind1 = int((validation_iter)*interval_size)
            end_ind1 = int((validation_iter+1)*interval_size)
            s_ind2 = int((validation_iter + 1) * interval_size)
            end_ind2 = int(no_examples)
        else:
            s_ind1 = 0
            end_ind1 = int((validation_iter)*interval_size)
            s_ind2 = int((validation_iter + 1) * interval_size)
            end_ind2 = int(no_examples)
        
        #print("train_ind ",s_ind1,end_ind1,s_ind2,end_ind2)
        indices = range(s_ind1,end_ind1) + range(s_ind2,end_ind2)
        c_train_data = c_fingerprints[indices]
        p_train_data = p_fingerprints[indices]
        labels_train = labels2[indices]
                               
    samples = np.random.randint(low=0,high=curr_data_size,size=(batch_size,1))
    if binary == True:
        train_batch = p_fingerprints[samples].reshape(batch_size,ip_dim)
        train_batch = train_batch.astype(int)
    else:
        train_batch = c_fingerprints[samples].reshape(batch_size,ip_dim2)
        train_batch = train_batch.astype(float)
    
    train_batch = torch.cuda.FloatTensor(train_batch)
    train_batch = Variable(train_batch,requires_grad=False).cuda()
    target = Variable(torch.cuda.LongTensor(labels_train[samples]),requires_grad=False)
    target = target.view(batch_size,)
    return train_batch,target

In [9]:
def get_val_data(validation_iter,binary = True):
    interval_size = int(no_examples)*0.2
    s_ind = int((validation_iter-1)*interval_size)
    e_ind = int((validation_iter) * interval_size)
    if(binary==True):
        train_data = p_fingerprints[s_ind:e_ind]
    else:
        train_data = c_fingerprints[s_ind:e_ind]
    labels_val = labels2[s_ind:e_ind]   
    #print("val ind ",s_ind,e_ind)
    return Variable(torch.cuda.FloatTensor(train_data)),labels_val  
        

In [13]:
class c_mlp(nn.Module):
    def __init__(self):
        super(c_mlp,self).__init__()
        self.l1 = nn.Linear(ip_dim2,100)
        self.l2 = nn.Linear(100,100)
        self.l4 = nn.Linear(100,50)
        self.l5 = nn.Linear(50,2)
        
    def forward(self,x,sse=False):
        f = []
        f1 = self.l1(x)
        self.save_for_backward(f1)
        #f.append(x)
        x = F.leaky_relu(f1)
        x = self.l2(x)
        f.append(x)
        x = F.leaky_relu(x)
        f.append(x)
        x = self.l4(x)
        f.append(x)
        x = F.leaky_relu(x)
        x = (self.l5(x))
        return x,f1.clone()

In [14]:
min_fn = 15
max_fp = 60
cm_list = []
for val_iter in range(1,6):
    print("val iter: ",val_iter)
    
   
    #weights_array = [5]
    weights_array = np.linspace(7,30,13)
    for i,w in enumerate(weights_array): 
        mymlp = c_mlp().cuda()
        optimizer = torch.optim.Adagrad(mymlp.parameters(),lr=1e-3)
        sse_optim = torch.optim.Adagrad(mymlp.parameters(),lr = 1e-3)
        criterion = nn.CrossEntropyLoss(weight=torch.cuda.FloatTensor([1,w]))

        for ep in range(3):
            print("ep:  ",ep)
            train_batch,target = get_train_batch(batch_size,binary = False,validation_iter = val_iter)
            model_op,f_op = mymlp(train_batch)
            
            loss = criterion(model_op,target)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
            

    #     if(ep%30==29):
    #         print(loss.data[0])
    
            t_int = (target.data.cpu().numpy())
            t_int = t_int.astype(int)
            #f_ac = f_op[t_int==1]
            #f_inac = f_op[t_int==0]
            #print(len(f_ac),len(f_inac))
            
            for k in range(len(f_op)):
                ind1 = torch.LongTensor(np.argwhere(t_int==0)).cuda()
                ind1 = ind1.view((ind1.size()[0],))
                #fnow = (f_op[k])
                f_ac = torch.index_select(f_op,dim=0,index = ind1)
                #compute only for 20-20 examples in each batch
                sse_loss = 0
                for i in range(20):
                    for j in range(i):
                        sse_loss  = sse_loss + torch.norm(f_ac[i]-f_ac[j])

                inac_ele = len(f_inac)

                for i in range(len(f_inac)):
                    for j in range(i):
                        sse_loss = sse_loss +  torch.norm(f_inac[i] - f_inac[j])

                for i in range(len(f_inac)):
                    for j in range(20):
                        sse_loss += torch.max(0, 2 - torch.norm(f_inac[i] - f_ac[j]))

                sse_loss = sse_loss/((20*(20-1)/2) * (inac_ele*(inac_ele-1)/2) * 20*f_inac)

                sse_loss.backward()
                sse_optim.step()
                sse_optim.zero_grad()

                #if ep%30 == 29:
                print sse_loss.data[0]
            
                    
        ## After training check on cross validation data
        val_data,labels_val = get_val_data(val_iter,binary = False)
        train_op = mymlp(val_data)
        train_op = train_op.cpu().data.numpy()
        pred_labels = np.argmax(train_op,axis=1)
        #tmp_labels = tmp_labels.data.cpu().numpy()
        #print(sum(tmp_labels))
        cf = metrics.confusion_matrix(labels_val,pred_labels).ravel()
        #print(val_iter,w)
        print('tn, fp, fn, tp: ',cf)
        [tn,fp,fn,tp] = cf
        wcf = [val_iter] + [w] + [cf]
        if(fn < 3):
            cm_list.append(wcf)
            if(fn<4):
                if(fp < max_fp):
                    max_fp = fp
                    model_path = os.getcwd() + '/cont_model_with_val_selu'
                    torch.save(mymlp.state_dict(),model_path)
                    print("saving model on val: ",val_iter," and weight: ",w)

        


('val iter: ', 1)
('ep:  ', 0)


AttributeError: 'c_mlp' object has no attribute 'save_for_backward'

In [22]:
f_op[0].type

<bound method Variable.type of Variable containing:
 1.9287e-01 -7.9549e-01  2.0571e-01  ...   9.5052e-02  3.9897e-01  1.1982e-02
-9.5301e-02 -4.3733e-01 -2.3684e-01  ...   5.4843e-01 -1.4905e-01 -2.4515e-01
-6.9170e-01 -1.8215e-01  3.9325e-01  ...  -1.6132e-01  1.3707e-01 -1.5613e-01
                ...                   ⋱                   ...                
 2.5191e-01 -7.0508e-01 -3.0295e-01  ...   2.2555e-01  5.3586e-01  1.3782e-01
 1.3518e-01 -9.6394e-01 -7.3459e-01  ...   1.1321e-01 -1.3200e-01 -1.5562e-01
 1.0098e-01 -1.9448e-01 -1.0422e-01  ...  -5.6309e-02  1.3173e-01 -1.8353e-01
[torch.cuda.FloatTensor of size 128x100 (GPU 0)]
>

In [15]:
# #weights_array = np.linspace(3,25,22)
# tmp_w = [weights_array[0]]
# cont_cm_lsit = []
# for i,w in enumerate(tmp_w):
    
    
#     c_mymlp = c_mlp().cuda()
#     optimizer2 = torch.optim.Adam(c_mymlp.parameters(),lr=1e-4)
    
#     weights = torch.cuda.FloatTensor([1,w])
#     criterion = nn.CrossEntropyLoss(weight=weights)
    
#     for ep in range(2000):
#         train_batch,target = get_train_batch(batch_size,binary=False)
#         #train_batch,target = sample_X(batch_size,labels='True')
#         #target = Variable(torch.cuda.LongTensor(target),requires_grad = False)
#         model_op = c_mymlp(train_batch)
#         #print(model_op.type)
#         #print(target.type)
#         loss = criterion(model_op,target)
#         loss.backward()
#         optimizer2.step()
#         optimizer2.zero_grad()
    
#         if(ep%100==99):
#             print(loss.data[0])

#     train_op = c_mymlp(Variable(torch.cuda.FloatTensor(c_fingerprints)))
#     train_op = train_op.cpu().data.numpy()
#     pred_labels = np.argmax(train_op,axis=1)
#     #tmp_labels = tmp_labels.data.cpu().numpy()
#     #print(sum(tmp_labels))
#     cont_cf = metrics.confusion_matrix(labels2,pred_labels).ravel()
#     print('tn, fp, fn, tp: ',cont_cf)
#     cont_cm_lsit.append(cont_cf)
    
#     #model_path = os.getcwd() + '/model' + str(i)
#     #torch.save(c_mymlp.state_dict(),model_path)
    

### Checking on Test data

In [13]:
path = os.getcwd()+'/bioassay-datasets/'
test_bin = []
test_con = []
labels = []
with open(path+'AID362red_test.csv') as csvfile:
    readcsv = csv.reader(csvfile)
    for row in readcsv:
        test_bin.append(row[:112])
        test_con.append(row[112:-1])
        labels.append(row[-1])

In [14]:
test_bin = np.asarray(test_bin)[1:]
test_bin = test_bin.astype(int)

test_con = np.asarray(test_con)[1:]
test_con = test_con.astype(float)

#Normalise the features
test_con = (test_con - np.mean(test_con,axis=0))/np.std(test_con,axis=0)

#p2_fingerprints = np.ones(p_fingerprints.shape)
(no_examples , ip_dim) = test_bin.shape
ip_dim2 = test_con.shape[1]
labels = labels[1:]
print(no_examples,ip_dim)
print(ip_dim2)
test_bin[(test_bin==0)] = -1


(856, 112)
32


In [15]:
best_c_mlp = c_mlp().cuda()
param_path = os.getcwd() + '/cont_model_with_val'
best_c_mlp.load_state_dict(torch.load(param_path))

KeyError: 'unexpected key "l3.weight" in state_dict'

In [ ]:
test_labels2 = np.zeros((len(labels),1))
for i,l in enumerate(labels):
    if l=='Active':
        test_labels2[i] = 1
    else:
        test_labels2[i] = 0
test_labels2 = test_labels2.astype(int)
total_pos = np.sum(test_labels2)
print("tot_positive",total_pos)

In [ ]:
test_op = best_c_mlp(Variable(torch.cuda.FloatTensor(test_con)))
test_op = test_op.cpu().data.numpy()
test_pred = np.argmax(test_op,axis=1)
    #tmp_labels = tmp_labels.data.cpu().numpy()
    #print(sum(tmp_labels))
t_cont_cf = metrics.confusion_matrix(test_labels2,test_pred).ravel()
print('tn, fp, fn, tp: ',t_cont_cf)